In [1]:
import magma as m
from loam.boards.icestick import IceStick

import mantle lattice ice40
import mantle lattice mantle40


![](./images/counter_diagram.jpg)

We will use the `DefineCircuit` pattern to define a counter module.

`DefineCircuit` takes as a firt argument the name of the circuit, and then an even number of arguments of the form `port1_name, port1_type, port2_name, port2_type, ...`. By using the python `*` operator, we can programmatically construct a list of argument names and types.

For this example we compose a list literal with the output of the `ClockInterface` function which produces a standard set of inputs for a clock, clock enable, and reset. In this case, we only include a clock (by default) and an optional clock enable `has_ce=has_ce`.

We instance an n-bit `Adders` and `Register` from the `mantle` library. We wire them up based on our counter diagram above. Finally, we call `m.wireclock` to automatically wire up the clock inputs to the `Register` instance.

In [2]:
from mantle import Add, Register


def DefineCounter(n, has_ce=False):

    name = "Counter{}{}".format(n, has_ce)

    args = ["O", m.Out(m.Bits(n)), "COUT", m.Out(m.Bit)] + m.ClockInterface(has_ce=has_ce)
    Counter = m.DefineCircuit(name, *args)

    add = Add(n, cin=False, cout=True)
    reg = Register(n, has_ce=has_ce)

    m.wire( reg.O, add.I0 )
    m.wire( m.bits(1, n), add.I1 )

    m.wire(add.O, reg.I)

    m.wire( reg.O, Counter.O )

    m.wire( add.COUT, Counter.COUT )

    m.wireclock(Counter, reg)
    m.wiredefaultclock(Counter, reg)

    m.EndDefine()
    return Counter

We can inspect the generated verilog

In [3]:
from magma.backend.verilog import compile as compile_verilog
print(compile_verilog(DefineCounter(4)))

compiling FullAdder
compiling Add4Cout
compiling Register4
compiling Counter4False
module FullAdder (input  I0, input  I1, input  CIN, output  O, output  COUT);
wire  inst0_O;
wire  inst1_CO;
SB_LUT4 #(.LUT_INIT(16'h9696)) inst0 (.I0(I0), .I1(I1), .I2(CIN), .I3(1'b0), .O(inst0_O));
SB_CARRY inst1 (.I0(I0), .I1(I1), .CI(CIN), .CO(inst1_CO));
assign O = inst0_O;
assign COUT = inst1_CO;
endmodule

module Add4Cout (input [3:0] I0, input [3:0] I1, output [3:0] O, output  COUT);
wire  inst0_O;
wire  inst0_COUT;
wire  inst1_O;
wire  inst1_COUT;
wire  inst2_O;
wire  inst2_COUT;
wire  inst3_O;
wire  inst3_COUT;
FullAdder inst0 (.I0(I0[0]), .I1(I1[0]), .CIN(1'b0), .O(inst0_O), .COUT(inst0_COUT));
FullAdder inst1 (.I0(I0[1]), .I1(I1[1]), .CIN(inst0_COUT), .O(inst1_O), .COUT(inst1_COUT));
FullAdder inst2 (.I0(I0[2]), .I1(I1[2]), .CIN(inst1_COUT), .O(inst2_O), .COUT(inst2_COUT));
FullAdder inst3 (.I0(I0[3]), .I1(I1[3]), .CIN(inst2_COUT), .O(inst3_O), .COUT(inst3_COUT));
assign O = {inst3_O,inst2_O,

To test our counter circuit, we will instance two versions. A 5-bit counter will be wired up to the icestick leds. Because the icestick clock runs too fast to see without a high-speed camera, we will wire up the carry out of a 23-bit counter to the clock enable of our 5-bit counter. This causes the 5-bit counter to advance slower (the clock enable will be asserted True for 1 cycle every time the 23-bit counter reaches it's maximum value).

In [4]:
icestick = IceStick()

icestick.Clock.on()
icestick.D1.on()
icestick.D2.on()
icestick.D3.on()
icestick.D4.on()
icestick.D5.on()

main = icestick.main()

counter4 = DefineCounter(5, has_ce=True)()
counter23 = DefineCounter(23)()
m.wire(counter23.COUT, counter4.CE)
m.wire(counter4.O, m.bits([main.D1, main.D2, main.D3, main.D4, main.D5]))

Compile and flash our design onto the icestick. The leds should blink in a 5-bit counter pattern. Change the size of the 23-bit control counter or replace it with a different circuit to experiment with the counting pattern.

In [5]:
m.compile("build/ice_counter", main)

compiling FullAdder
compiling Add5Cout
compiling Register5CE
compiling Counter5True
compiling Add23Cout
compiling Register23
compiling Counter23False
compiling main


In [6]:
%%bash
cd build
yosys -q -p 'synth_ice40 -top main -blif ice_counter.blif' ice_counter.v
arachne-pnr -q -d 1k -o ice_counter.txt -p ice_counter.pcf ice_counter.blif
icepack ice_counter.txt ice_counter.bin

iceprog ice_counter.bin

init..
cdone: high
reset..
cdone: low
flash ID: 0x20 0xBA 0x16 0x10 0x00 0x00 0x23 0x51 0x73 0x10 0x23 0x00 0x35 0x00 0x35 0x06 0x06 0x15 0x43 0xB6
file size: 32220
erase 64kB sector at 0x000000..
programming..
reading..
VERIFY OK
cdone: high
Bye.


In [7]:
m.compile("build/ice_counter", main, output="coreir")

In [8]:
%%bash
cd build
cat ice_counter.json
coreir --load_libs libcoreir-ice40.dylib -i ice_counter.json -o ice_counter_coreir.v
yosys -q -p 'synth_ice40 -top main -blif ice_counter_coreir.blif' ice_counter_coreir.v
arachne-pnr -q -d 1k -o ice_counter_coreir.txt -p ice_counter.pcf ice_counter_coreir.blif
icepack ice_counter_coreir.txt ice_counter_coreir.bin

iceprog ice_counter_coreir.bin

{"top":"global.main",
"namespaces":{
  "global":{
    "modules":{
      "Add23Cout":{
        "type":["Record",[
          ["I0",["Array",23,"BitIn"]],
          ["I1",["Array",23,"BitIn"]],
          ["O",["Array",23,"Bit"]],
          ["COUT","Bit"]
        ]],
        "instances":{
          "bit_const_GND":{
            "modref":"corebit.const",
            "modargs":{"value":["Bool",false]}
          },
          "inst0":{
            "modref":"global.FullAdder"
          },
          "inst1":{
            "modref":"global.FullAdder"
          },
          "inst10":{
            "modref":"global.FullAdder"
          },
          "inst11":{
            "modref":"global.FullAdder"
          },
          "inst12":{
            "modref":"global.FullAdder"
          },
          "inst13":{
            "modref":"global.FullAdder"
          },
          "inst14":{
            "modref":"global.FullAdder"
          },
          "inst15":{
            "modref":"global.FullAdder"
          }

ERROR: Cannot find lib libcoreir-ice40.dylib

0   coreir                              0x000000010ea9370c _Z8parseLibNSt3__112basic_stringIcNS_11char_traitsIcEENS_9allocatorIcEEEERS5_S6_RNS_6vectorIS5_NS3_IS5_EEEE + 11484
1   coreir                              0x000000010ea9af49 main + 24409
2   libdyld.dylib                       0x00007fff63e73015 start + 1
ERROR: Can't open input file `ice_counter_coreir.v' for reading: No such file or directory
fatal error: read_blif: failed to open `ice_counter_coreir.blif': No such file or directory
Error: Failed to open input file.
iceprog: can't open 'ice_counter_coreir.bin' for reading: No such file or directory
